# ipy widgets in Jupyter Notebook
* Code shows how figurewidget app works
* Ref: [plolty - figurewidget app](https://plotly.com/python/figurewidget-app/)

## Note:
This widget works in standalone Jupyter Notebook. But it [does not work in Jupyter lab](https://github.com/jupyter-widgets/ipywidgets/issues/2623#issuecomment-579601758) !!! 

In [1]:
import datetime
import numpy as np
import pandas as pd

import plotly.io as pio
pio.renderers.default = 'iframe'

import plotly.graph_objects as go
from ipywidgets import widgets

In [2]:
# get the data
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [3]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
183386,2013,4,20,853.0,-2.0,1040.0,-29.0,DL,N774NC,2239,EWR,DTW,85.0,488,8.0,53.0
95757,2013,12,14,1448.0,2.0,1830.0,37.0,B6,N708JB,325,JFK,TPA,166.0,1005,14.0,48.0
18263,2013,1,22,616.0,16.0,939.0,74.0,MQ,N508MQ,4650,LGA,ATL,128.0,762,6.0,16.0


Let us get the set of all airlines, so that we can type the right things into the search box later.

In [4]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

Let us assign the widgets we are going to use in our app. In general all these widgets will be used to filter the data set for visualization

In [5]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description= 'Date: ',
    value=True
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)

# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')


g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

Let us now write a function that will handle the input from the widgets and alter the graph state

In [6]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False
    
def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]
        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        
        with g.batch_update():
            g.date[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.axis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'
            
origin.observe(response, names='value')
textbox.observe(response, names='value')
month.observe(response, names='value')
use_date.observe(response, names='value')

Let us try out the app!!

In [7]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])